In [1]:
import numpy as np
from queue import PriorityQueue as queue
from collections import defaultdict
from faker import Faker
from tabulate import tabulate
from collections import deque, Counter

fake = Faker()

## Draw Simulation Methods

In [2]:
def init_players(num_players):
    """
    Create a collection of players in the form [player_id, player_name, player_ranking]
    Arguments:
        num_players: Number of players to add to the draw
    Returns:
        players: Array of player details
            (n x 3) list containing [player_id, player_name, player_ranking]
    """
    players = []
    id_to_name, id_to_rank = defaultdict(lambda: 'Bye'), defaultdict(lambda: 1600)
    for n in range(num_players):
        p_id, p_name, p_rank = np.random.randint(1000000, 9999999), fake.name(), np.random.randint(1600, 2000)
        players.append([p_id, p_name, p_rank])
        id_to_name[p_id] = p_name
        id_to_rank[p_id] = p_rank
    return players, id_to_name, id_to_rank

In [3]:
def init_pairings(players):
    """
    Randomly create results for the first round
    Arguments:
        players: (n x 3) numpy array containing [player_id, player_name, player_ranking]
    Returns:
        results: Randomly generated results table
            (n x 3) list containing [player_a_id, player_b_id, match_winner_id]
    """
    results = []
    for n in range(0, len(players), 2):
        a, b = players[n][0], players[n+1][0]
        result = [a, b, np.random.choice([a, b, None], 1, p=(0.45, 0.45, 0.1))[0]]
        results.append(result)
    return results

In [4]:
def run_round(pairings, rnd, p_draw=0):
    """
    Randomly create results for a round
    Arguments:
        pairings: List of pairings for the round to be simulated
            (n x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    Returns:
        results: Randomly generated results table
            (n x 3) list containing [player_a_id, player_b_id, match_winner_id]
    """
    results = []
    p_win = (1 - p_draw) / 2
    BYE = 0
    for pairing in pairings:
        a, b, pod = pairing[0][1], pairing[1][1], pairing[2]
        if a <= BYE:
            result = [a,b,b]
        elif b <= BYE:
            result = [a,b,a]
        else:
            result = [a, b, np.random.choice([a, b, None], 1, p=(p_win, p_win, p_draw))[0]]
        results.append(result + [rnd, pod])
        
    return results

## Draw Calculation Methods

In [5]:
def create_draw(players, results, method='bracket'):
    """
    Create draw for the round
    Arguments:
        players: Array of player details
            (n x 3) list containing [player_id, player_name, player_ranking]
        results: List of pairings and results for all rounds to date
            (m x 2) list containing [player_a_id, player_b_id, winner_id]
    Returns:
        standings: List of standings for all players still in the tournament
            (n x 2) list containing [player_id, player_wins]
        splits: List of index points where each points bracket finishes
        scores: Matrix containing the scores for all possible pairings
            (n x n) matrix of floats
        alloc: The matrix form pairing allocations
            (n x n) matrix of integers
        pairings: The list form of pairing allocations
            (n / 2 x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    """
    assert method in ['bracket', 'create pod', 'reuse pod']
    n_players = len(players)
    
    standings       = get_standings(players, results, method)
    scores, splits  = get_scores(standings, results)
    alloc, pairings = get_allocations(standings, scores)
    
    return standings, splits, scores, alloc, pairings

In [6]:
def get_standings(players, results, method):
    """
    Calculate player standings at start of round
    Arguments:
        results: List of pairings and results for all rounds to date
            (m x 2) list containing [player_a_id, player_b_id, winner_id]
    Returns:
        standings: List of standings for all players still in the tournament
            (n x 3) list containing [player_id, player_wins, player_pod]
    """
    assert method in ['bracket', 'create pod', 'reuse pod']

    standings, pods = defaultdict(int), defaultdict(lambda: 1)
    active_players  = set([player[0] for player in players])
    
    # Calculate player standings
    if len(results) > 0:
        last_round = np.array(results)[:,3].max()
        for a, b, winner, rnd, pod in results:
            standings[a] += (a == winner)
            standings[b] += (b == winner)
            if rnd == last_round:
                if a in active_players: pods[a] = pod
                if b in active_players: pods[b] = pod
    
    standings = [[p, standings[p]] for p in active_players]
    standings = sorted(standings, key=lambda x: x[1], reverse=True)
    
    # Allocate players to pods
    for pos, standing in enumerate(standings):
        if method == 'create pod':
            standing += [pos // 8 + 1]
        elif method == 'reuse pod':
            standing += [pods[standing[0]]]
        else:
            standing += [1]
    
    standings = np.array(standings)
            
    # Add byes into the largest bracket in each of the pods
    pod_sizes = Counter(standings[:,-1])
    byes = []
    for pod_num in range(1, len(pod_sizes) + 1):
        if pod_sizes[pod_num] % 2 == 1:
            mode = np.bincount(standings[standings[:,2]==pod_num][:,1]).argmax()
            byes.append([-pod_num, mode, pod_num])
    
    if len(byes) > 0:
        standings = np.vstack([standings, byes])
        standings = sorted(standings, key=lambda x: x[1], reverse=True)
        standings = sorted(standings, key=lambda x: x[2])
        standings = np.array(standings)
    
    return standings

In [7]:
def get_scores(standings, results):
    """
    Assign scores to each possible pairing
    A higher score means that the pairing is more strongly preferred
    Arguments:
        standings: List of standings for all players still in the tournament
            (n x 2) list containing [player_id, player_wins]
        pairs: List of pairings for all rounds to date
            (m x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    Returns:
        scores: Matrix containing the scores for all possible pairings
            (n x n) matrix of floats
        splits: List of index points where each points bracket finishes
    """
    n_players = len(standings)
    ix = {player_id: pos for pos, player_id in enumerate(standings[:,0])}
    active_players = set(ix.keys())
    splits = get_splits(standings)
    
    # Calculate scores for pairings
    scores = np.full((n_players, n_players), -100)
    
    split_1, split_2 = 0, 0
    for split in splits + [n_players]:
        # Assign points for players in same bracket
        scores[split_1:split, split_1:split] = 16
        # Assign points for pull-up bracket
        if split_1 %2 == 1:
            scores[split_2:split_1, split_1:split] = 1
            scores[split_1:split, split_2:split_1] = 1
        split_1, split_2 = split, split_1
    
    # Reset self vs self to -inf
    np.fill_diagonal(scores, -100)

    # Subtract points for prior matchup
    BYE = 0
    for result in results:
        if result[0] in active_players and result[1] in active_players:
            a, b = ix[result[0]], ix[result[1]]
            penalty = 32 if min(result) < BYE else 8
            scores[a,b] -= penalty
            scores[b,a] -= penalty
        
    return scores, splits

In [8]:
def swap_pairs(pair_1, pair_2, alloc):
    """
    Helper function to swap players between two pairings
    Original pairing of (a,b) and (c,d) becomes (a,c) and (b,d)
    Arguments:
        pair_1: Tuple containing first pair to swap (a,b)
        pair_2: Tuple containing first pair to swap (c,d)
        alloc: The matrix form pairing allocations
            (n x n) matrix of integers
    Returns:
        alloc: The matrix form pairing allocations with [(a,b), (c,d)] -> [(a,c), (b,d)]
            (n x n) matrix of integers        
    """
    a, b = pair_1
    c, d = pair_2

    alloc[a, b] = 0
    alloc[b, a] = 0
    alloc[c, d] = 0
    alloc[d, c] = 0
    alloc[a, c] = 1
    alloc[c, a] = 1
    alloc[b, d] = 1
    alloc[d, b] = 1
    
    return alloc

In [9]:
def get_splits(standings):
    """
    Helper method to find the points at which the brackets split
    Arguments:
        standings: List of standings for all players still in the tournament
            (n x 3) list containing [player_id, player_wins, bracket]
    """
    n_players = len(standings)
    
    # Sort standings by bracket and player wins
    standings = sorted(standings, key=lambda x: x[1], reverse=True)
    standings = sorted(standings, key=lambda x: x[2])
    standings = np.array(standings)
    
    # Calculate splits
    splits = [n for n in range(n_players) if (standings[n,[1,2]] != standings[n-1,[1,2]]).any()] + [n_players]

    return splits

In [10]:
def get_allocations(standings, scores):
    """
    Split table down into even length brackets and sub-allocate
    Arguments:
        standings: List of standings for all players still in the tournament
            (n x 2) list containing [player_id, player_wins]
        scores: Matrix containing the scores for all possible pairings
            (n x n) matrix of floats
    Returns:
        alloc: The matrix form pairing allocations
            (n x n) matrix of integers
        pairs: The list form of pairing allocations
            (n / 2 x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    """
    n_players = len(standings)
    splits = get_splits(standings)

    # Allocate entire table if there is no split
    if len(splits) == 1:
        return _get_allocations(standings, scores)

    allocs = np.zeros((n_players, n_players))
    pairs = []

    # Split into sub-tables at even numbered index split points
    ixs = np.array(splits)
    ixs = ixs[ixs % 2 == 0]
    for ix1, ix2 in zip(ixs[:-1], ixs[1:]):
        alloc, pair = _get_allocations(standings[ix1:ix2], scores[ix1:ix2,ix1:ix2], start=ix1)

        allocs[ix1:ix2,ix1:ix2] = alloc
        pairs += pair

    return allocs, pairs

def _get_allocations(standings, scores, start=0):
    """
    Allocate player pairings
    Arguments:
        standings: List of standings for all players still in the tournament
            (n x 2) list containing [player_id, player_wins]
        scores: Matrix containing the scores for all possible pairings
            (n x n) matrix of floats
    Returns:
        alloc: The matrix form pairing allocations
            (n x n) matrix of integers
        pairs: The list form of pairing allocations
            (n / 2 x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    """
    n_players = len(standings)
    ix = {pos+start: player_id for pos, player_id in enumerate(standings[:,0])}
    alloc = np.zeros((n_players, n_players), dtype=int)
    splits = get_splits(standings)
    pods = {player[0]: player[2] for player in standings}

    # Construct initial draw
    for _ in range(int(n_players / 2)):
        mask = (1 - alloc.sum(axis=1))
        # Choose Player A
        p = scores.max() - scores.mean(axis=1)
        p = p * mask
        a = p.argmax()
        # Choose Player B
        p = np.exp(scores[a])
        p = np.maximum(p * mask, 0)
        b = np.random.choice(range(n_players),p=p/p.sum())
        
        alloc[a, b] = 1
        alloc[b, a] = 1

    score_q = [(alloc*scores).sum()/len(scores)]
    max_score = 16 - 15 * (np.array(splits) % 2).sum() * 2 / len(scores)
    
    # Run Metropolis-Hastings to iteratively improve draw
    if score_q[0] != max_score:
        prob = np.exp(scores)
        prob /= prob.sum(axis=1, keepdims=True)

        best_alloc = alloc.copy()
        while len(score_q) < 10 or np.mean(score_q[-5:]) > np.mean(score_q[-10:-5]):        
            if max(score_q) == max_score:
                break
            for a, p in enumerate(prob):
                b = alloc[a].argmax()
                c = np.random.choice(range(len(p)),p=p)
                d = alloc[c].argmax()
                p_change = prob[a,c]*prob[b,d]/(prob[a,b]*prob[c,d] + prob[a,c]*prob[b,d])
                if np.random.random() < p_change:
                    alloc = swap_pairs((a,b), (c,d), alloc)

            score = (alloc*scores).sum()/len(prob)
            if score > max(score_q):
                best_alloc = alloc.copy()
            score_q.append(score)

        alloc = best_alloc
        del(best_alloc)
    
    # Create list form of pairings
    alloc_ = alloc * np.tri(*alloc.shape)

    pairs = []
    for n in range(n_players):
        if alloc_[n].max() == 1:
            a, b = n + start, alloc_[n].argmax() + start
            pairs.append([(a, ix[a]), (b, ix[b]), pods[ix[a]]])
            
    return alloc, pairs

## Draw Printing Methods

In [35]:
def print_standings(standings, id_to_name):
    """
    Print standings for the round in table form
    """
    table = []
    for rank, player in enumerate(standings):
        table.append([int(player[0]), id_to_name[player[0]], player[1], rank+1, player[2]])
    print('\n', tabulate(table, headers=['ID', 'Name', 'Wins', 'Rank', 'Pod']), '\n')
    

def print_pairings(pairs, id_to_name, scores):
    """
    Print pairings for the round in table form
    """
    table = []
    for table_num, pair in enumerate(pairs):
        b, a, pod = pair[0], pair[1], pair[2]
        score = scores[b[0], a[0]]
        row = [table_num+1, pod, id_to_name[a[1]], int(a[1]), int(a[0])+1, id_to_name[b[1]], int(b[1]), int(b[0])+1, score]
        table.append(row)
    headers = ['Table #', 'Pod #', 'Player 1 - Name', 'ID', 'Rank', 'Player 2 - Name', 'ID', 'Rank', 'GOF']
    print('\n', tabulate(table, headers=headers), '\n')

def print_byes(pairs):
    byes = []
    for pairs in pairs_h:
        for pair in pairs:
            a, b = pair[0][1], pair[1][1]
            if min(a, b) < 0:
                byes += [max(a, -1), max(b, -1)]
    byes = Counter(np.reshape(byes, -1))

    print('----- BYES -----')
    for k, v in byes.items():
        if k < 0:
            print('Byes', ': ', v)
        else:
            print(k, ': ', v)

In [32]:
Counter().update([1])

## Run a sample Tournament
Create a list of players then run through a multi-round tournament and print the results
Each round prints:
- Goodness of fit score: Score for the pairings allocated in the round. The maximum possible score is 4. If a number < 0 comes out, something has probably gone wrong
- Standings: A table of standings of the players (based only on # of wins, no accounting for draws, op wins etc)
- Pairings: A table of pairings for the following round (including player name and standing so you can check if players are being paired with others nearby)

To show all the standings and pairings at once, make sure to select the cell, click on the "Cell" menu, select -> "Current Outputs" and then select "Toggle Scrolling"

The entire history of "standings", "splits", "scores", "allocations", and "pairs" is recorded in the lists *standings_h, splits_h, scores_h, alloc_h, pairs_h*
    
Note: Early stage demo only. This does not support:
- Odd numbers of players and byes
- Allocating pods
- Iterative sampling methods to improve draw quality
- etc

In [45]:
NUMBER_OF_PLAYERS = 63
ROUNDS  = ['bracket', 'bracket', 'bracket',
           'create pod', 'reuse pod', 'reuse pod',
           'create pod', 'reuse pod', 'reuse pod']
NUMBER_OF_ROUNDS = len(ROUNDS)
PROBABILITY_OF_DRAW = 0.0001

In [46]:
# Create players
players, id_to_name, id_to_rank = init_players(NUMBER_OF_PLAYERS)

In [47]:
results = []
standings_h, splits_h, scores_h, alloc_h, pairs_h = [], [], [], [], []
number_of_players = NUMBER_OF_PLAYERS + NUMBER_OF_PLAYERS % 2

for rnd, method in enumerate(ROUNDS):
    standings, splits, scores, alloc, pairs = create_draw(players, results, method)
    for collection, value in zip([standings_h, splits_h, scores_h, alloc_h, pairs_h],
                                 [standings,   splits,   scores,   alloc,   pairs]):
        collection.append(value)
    print('----- Round', rnd + 1, '-----', )
    print('Goodness of Fit:', (scores * alloc).sum() / number_of_players)
    print('Maximum GOF:', 16 - 15 * (np.array(splits) % 2).sum() * 2 / number_of_players)
    print("Frequencies: " + ', '.join([str(int(k)) + "pts: " + str(int(v / 2)) 
                                 for k, v in Counter((scores * alloc).reshape(-1)).items() if k != 0]) )
    print_standings(standings, id_to_name, )
    print_pairings(pairs, id_to_name, scores)
    result = run_round(pairs, rnd, p_draw=PROBABILITY_OF_DRAW)
    results += result

print('Overall Pairing Score Frequency: ', ', '.join([str(int(k)) + "pts: " + str(int(v / 2))
   for k, v in Counter((np.array(scores_h)*np.array(alloc_h))[-NUMBER_OF_ROUNDS:].reshape(-1)).items() if k != 0]))
print_byes(pairs_h)

----- Round 1 -----
Goodness of Fit: 16.0
Maximum GOF: 16.0
Frequencies: 16pts: 32

      ID  Name                     Wins    Rank    Pod
-------  ---------------------  ------  ------  -----
5996160  Robert Kim                  0       1      1
5391744  Rebecca Cardenas            0       2      1
6359939  Diana Trevino               0       3      1
7711876  Emily Hunt                  0       4      1
5493765  Edward Rose                 0       5      1
1994246  Xavier Watkins              0       6      1
5515019  John Reeves                 0       7      1
3982348  Connor Smith                0       8      1
7938573  Amber Osborne               0       9      1
6603147  Eric Diaz                   0      10      1
6513553  Joanna Woodward             0      11      1
5309463  Bianca Taylor               0      12      1
1231899  Neil Boyd                   0      13      1
2000797  Nicole Wagner               0      14      1
6096034  Deborah Solis               0      15     

----- Round 6 -----
Goodness of Fit: 16.0
Maximum GOF: 16.0
Frequencies: 16pts: 32

      ID  Name                     Wins    Rank    Pod
-------  ---------------------  ------  ------  -----
5309463  Bianca Taylor               5       1      1
3921395  Devon Charles               5       2      1
5996160  Robert Kim                  4       3      1
5478487  Clayton Walker              4       4      1
6850777  Ricky King                  4       5      1
4713703  David Scott                 4       6      1
7376830  Lynn Jones                  3       7      1
8264052  Shirley Flores              3       8      1
6359939  Diana Trevino               4       9      2
1994246  Xavier Watkins              4      10      2
5391744  Rebecca Cardenas            3      11      2
7711876  Emily Hunt                  3      12      2
6349858  Leah Hernandez              3      13      2
4127143  Ronald Cook                 3      14      2
2000797  Nicole Wagner               2      15     

Overall Pairing Score Frequency:  16pts: 264, 1pts: 21, 8pts: 3
----- BYES -----
Byes :  9
4127143 :  1
5391744 :  1
6243525 :  1
7442361 :  1
1824977 :  1
6513553 :  1
8014454 :  1
4368573 :  1
4005177 :  1
